In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install mne
! pip install scipy
! pip install scikit-learn
! pip install mne-icalabel
! pip -q install -U eeglabio mne mne-icalabel
import numpy as np
import scipy.io
import mne
from mne.preprocessing import ICA
from mne_icalabel import label_components
from mne import annotations_from_events
from sklearn.linear_model import LinearRegression
import os, json, sys

In [3]:
!ls /content/drive/MyDrive/Project_Code


Concatenate_all.ipynb			preprocessing_copy.py
Concatenate_all.ipynb（副本）		Preprocessing.py
Concat_one_test.ipynb			Preprocess.ipynb
Extract_features_one_participant.ipynb	__pycache__
Preprocess_eeglab_event.ipynb		Single_preprocess.ipynb
Preprocess_eeglab.ipynb			Untitled0.ipynb
Preprocessing_all.ipynb


In [4]:

import sys
sys.path.append("/content/drive/MyDrive/Project_Code")
import Preprocessing
from mne.export import export_raw


In [ ]:

from Preprocessing import (
    detect_bad_channels,
    detected_bad_line_noise,
    detect_flatline_channels,
    detect_bad_channels_std,
    assess_quality,
)

input_dir  = '/content/drive/MyDrive/concat_raws'
output_dir = '/content/drive/MyDrive/preprocessed_data_set'
os.makedirs(output_dir, exist_ok=True)

outer_electrodes = [
    "E1","E8","E14","E17","E21","E25","E32","E48","E49","E56",
    "E63","E68","E73","E81","E88","E94","E99","E107","E113",
    "E119","E125","E126","E127","E128"
]

bad_list, good_list = [], []
artifact_classes = ['eye blink', 'muscle', 'heart', 'line noise', 'channel noise']

for fname in sorted(os.listdir(input_dir)):
    if not fname.lower().endswith('.fif'):
        continue


    in_path  = os.path.join(input_dir, fname)
    base     = os.path.splitext(fname)[0]
    out_fif  = os.path.join(output_dir, f"{base}_clean.fif")

    try:
        raw = mne.io.read_raw_fif(in_path, preload=True, verbose='ERROR')
        events = mne.find_events(raw, stim_channel='STI', verbose=False)
        event_id = {int(e): str(e) for e in np.unique(events[:, 2])}
        # 1) bad channels
        print("detecting bad cahnnels")
        bads_corr = detect_bad_channels(raw, thresh=0.85)
        bads_ln, _ = detected_bad_line_noise(raw, line_freq=50, thresh=4)
        flat_bads  = detect_flatline_channels(raw, flat_sec=5)
        bads_all   = list(set(bads_corr + bads_ln + flat_bads))
        print("detecting bad channels done")
        # 2) montag + interpolation
        mont = mne.channels.make_standard_montage('GSN-HydroCel-128', head_size="auto")
        raw.set_montage(mont, on_missing='ignore')
        raw.info["bads"] = bads_all
        raw.interpolate_bads(reset_bads=True)

        # 3) filter
        raw = raw.filter(l_freq=2, h_freq=None, picks='eeg')
        raw = raw.notch_filter(freqs=[50, 100, 150, 200], picks='eeg')

        # 4) ICA + ICLabel
        raw_ica = raw.copy().filter(l_freq=2, h_freq=None, picks='eeg')
        ica = ICA(n_components=0.99, method='fastica', random_state=824, max_iter=800)
        ica.fit(raw_ica)
        out = label_components(raw_ica, ica, method='iclabel')
        labels, proba = out['labels'], out['y_pred_proba']
        bad_idx = []
        for ic, (label, prob) in enumerate(zip(labels, proba)):
          if label in artifact_classes and prob > 0.8:
            bad_idx.append(ic)
        ica.exclude = bad_idx
        raw_clean = raw.copy()
        ica.apply(raw_clean)
        print("ICA done")
        # 5) ICA bad channels + interpolation
        bads_std = detect_bad_channels_std(raw_clean, std_thresh=25e-6, artifact_thresh=100e-6, max_loss=0.5)
        raw_clean.info['bads'] = bads_std
        raw_clean.interpolate_bads(reset_bads=True)

        # 6) assess quality
        is_bad, reasons = assess_quality(raw_clean)


        annotations = annotations_from_events(
            events=events,
            sfreq=raw_clean.info['sfreq'],
            event_desc=event_id
        )
        raw_clean.set_annotations(annotations)

        # drop STI
        if 'STI' in raw_clean.ch_names:
            raw_clean.drop_channels(['STI'])
        print(f"{fname} → annotations 已写入，STI 通道已删除")
        print(f"numberof event: {len(events)}")
        if is_bad:
            print(f"{fname} → failed：{reasons}")
            bad_list.append(f"{fname}（reason：{reasons}）")
        else:
            raw_clean.drop_channels([ch for ch in outer_electrodes if ch in raw_clean.ch_names])
            raw_clean.apply_function(lambda x: x * 1e6, picks='eeg', channel_wise=True)
            out_set = os.path.join(output_dir, f"{base}_clean.set")
            mne.export.export_raw(out_set, raw_clean, fmt='eeglab', overwrite=True)
            good_list.append(fname)

        print("\nGOOD file：", good_list)
        print("BAD  files：", bad_list)

    except Exception as e:
      bad_list.append(f"{fname}（error：{e}）")
      print(f"{fname} → failed：{e}")


print("\nGOOD files：", len(good_list))
print("BAD  files：", len(bad_list))

detecting bad cahnnels
Effective window size : 4.096 (s)
finish 0
finish 1
finish 2
finish 3
finish 4
finish 5
finish 6
finish 7
finish 8
finish 9
finish 10
finish 11
finish 12
finish 13
finish 14
finish 15
finish 16
finish 17
finish 18
finish 19
finish 20
finish 21
finish 22
finish 23
finish 24
finish 25
finish 26
finish 27
finish 28
finish 29
finish 30
finish 31
finish 32
finish 33
finish 34
finish 35
finish 36
finish 37
finish 38
finish 39
finish 40
finish 41
finish 42
finish 43
finish 44
finish 45
finish 46
finish 47
finish 48
finish 49
finish 50
finish 51
finish 52
finish 53
finish 54
finish 55
finish 56
finish 57
finish 58
finish 59
finish 60
finish 61
finish 62
finish 63
finish 64
finish 65
finish 66
finish 67
finish 68
finish 69
finish 70
finish 71
finish 72
finish 73
finish 74
finish 75
finish 76
finish 77
finish 78
finish 79
finish 80
finish 81
finish 82
finish 83
finish 84
finish 85
finish 86
finish 87
finish 88
finish 89
finish 90
finish 91
finish 92
finish 93
finish 94
fin

/tmp/ipython-input-1787851394.py:46: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads(reset_bads=True)


Setting up high-pass filter at 2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Filter length: 825 samples (1.650 s)

Filtering raw data in 10 contiguous segments
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3301 samples (6.602 s)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) me

/tmp/ipython-input-1787851394.py:56: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  out = label_components(raw_ica, ica, method='iclabel')
/tmp/ipython-input-1787851394.py:56: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  out = label_components(raw_ica, ica, method='iclabel')
/tmp/ipython-input-1787851394.py:56: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use the extended infomax algorithm, use the 'mne.preprocessing.ICA' instance with the arguments 'ICA(method=

Applying ICA to Raw instance
    Transforming to ICA space (18 components)
    Zeroing out 2 ICA components
    Projecting back using 128 PCA components
ICA done
Setting channel interpolation method to {'eeg': 'spline'}.


...

YMD_concat_raw.fif → annotations 已写入，STI 通道已删除
numberof event: 1552

GOOD file： ['YAC_concat_raw.fif', 'YAG_concat_raw.fif', 'YAK_concat_raw.fif', 'YDG_concat_raw.fif', 'YDR_concat_raw.fif', 'YFR_concat_raw.fif', 'YFS_concat_raw.fif', 'YHS_concat_raw.fif', 'YIS_concat_raw.fif', 'YLS_concat_raw.fif', 'YMD_concat_raw.fif']
BAD  files： []
detecting bad cahnnels
Effective window size : 4.096 (s)
finish 0
finish 1
finish 2
finish 3
finish 4
finish 5
finish 6
finish 7
finish 8
finish 9
finish 10
finish 11
finish 12
finish 13
finish 14
finish 15
finish 16
finish 17
finish 18
finish 19
finish 20
finish 21
finish 22
finish 23
finish 24
finish 25
finish 26
finish 27
finish 28
finish 29
finish 30
finish 31
finish 32
finish 33
finish 34
finish 35
finish 36
finish 37
finish 38
finish 39
finish 40
finish 41
finish 42
finish 43
finish 44
finish 45
finish 46
finish 47
finish 48
finish 49
finish 50
finish 51
finish 52
finish 53
finish 54
finish 55
finish 56
finish 57
finish 58
finish 59
finish 60
finis

/tmp/ipython-input-1787851394.py:46: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads(reset_bads=True)


Setting up high-pass filter at 2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Filter length: 825 samples (1.650 s)

Filtering raw data in 5 contiguous segments
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3301 samples (6.602 s)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) met

/tmp/ipython-input-1787851394.py:56: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  out = label_components(raw_ica, ica, method='iclabel')
/tmp/ipython-input-1787851394.py:56: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  out = label_components(raw_ica, ica, method='iclabel')
/tmp/ipython-input-1787851394.py:56: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use the extended infomax algorithm, use the 'mne.preprocessing.ICA' instance with the arguments 'ICA(method=

Applying ICA to Raw instance
    Transforming to ICA space (17 components)
    Zeroing out 5 ICA components
    Projecting back using 128 PCA components
ICA done
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 94.2 mm
Computing interpolation matrix from 126 sensor positions
Interpolating 2 sensors
YMS_concat_raw.fif → annotations 已写入，STI 通道已删除
numberof event: 590

GOOD file： ['YAC_concat_raw.fif', 'YAG_concat_raw.fif', 'YAK_concat_raw.fif', 'YDG_concat_raw.fif', 'YDR_concat_raw.fif', 'YFR_concat_raw.fif', 'YFS_concat_raw.fif', 'YHS_concat_raw.fif', 'YIS_concat_raw.fif', 'YLS_concat_raw.fif', 'YMD_concat_raw.fif', 'YMS_concat_raw.fif']
BAD  files： []
detecting bad cahnnels
